**Data Processing for Car-Dheko Used Car Price Prediction**

In [ ]:
import pandas as pd
import os
from glob import glob

# 1. Specify the correct folder path (where your Excel files are stored)
folder_path = "csv_files"  # Replace with your actual folder name if different

# 2. List all city Excel files in the folder
city_files = glob(os.path.join(folder_path, '*.xlsx'))  # Searches inside 'csv_files' folder
print(f"Found Excel files in '{folder_path}':", city_files)

if not city_files:
    print(f"\n❌ No Excel files found in '{folder_path}'. Please check:")
    print("- Folder name is correct (case-sensitive)")
    print("- Files have '.xlsx' extension")
    print("- Files are not in a sub-subfolder")
else:
    # 3. Create a summary DataFrame
    summary_data = []

    for file in city_files:
        city_name = os.path.splitext(os.path.basename(file))[0]  # Extract filename without path/extension
        try:
            # Read first 2 rows to check structure
            df_sample = pd.read_excel(file, nrows=2)
            
            # Get file info
            file_size = f"{round(os.path.getsize(file)/(1024*1024), 2)} MB"
            file_columns = df_sample.columns.tolist()
            file_shape = pd.read_excel(file).shape
            
            summary_data.append({
                'City': city_name,
                'File_Type': 'Excel (XLSX)',
                'File_Size': file_size,
                'Rows': file_shape[0],
                'Columns': file_shape[1],
                'Columns_List': file_columns,
                'Sample_First_Row': df_sample.iloc[0].to_dict() if len(df_sample) > 0 else {}
            })
        except Exception as e:
            print(f"Error reading {file}: {str(e)}")
            summary_data.append({
                'City': city_name,
                'Error': str(e)
            })

    # 4. Create summary DataFrame
    summary_df = pd.DataFrame(summary_data)

    # 5. Display results
    if not summary_df.empty:
        print("\n📊 File Summary:")
        print(summary_df[['City', 'File_Type', 'File_Size', 'Rows', 'Columns']])
        
        # Save to Excel
        output_path = os.path.join(folder_path, 'city_files_summary.xlsx')
        summary_df.to_excel(output_path, index=False)
        print(f"\n✅ Summary saved to: {output_path}")
    else:
        print("\n❌ All files failed to load. Check error messages above.")

Found Excel files: []

Available columns in summary_df: []

No valid data to display. All files failed to load.

No data to save - all files failed to load.


In [ ]:
# Test reading one file
test_file = city_files[0]  # or specify a filename
try:
    test_df = pd.read_excel(test_file)
    print(f"Successfully read {test_file}")
    print("Columns:", test_df.columns.tolist())
except Exception as e:
    print(f"Failed to read {test_file}: {str(e)}")

In [ ]:
# Enhanced Version (Checks All Sheets):
# If your Excel files have multiple sheets, use this:

# Modified loop to check all sheets
for file in city_files:
    city_name = os.path.splitext(file)[0]
    try:
        xl = pd.ExcelFile(file)  # Open Excel file
        for sheet_name in xl.sheet_names:  # Loop through all sheets
            df_sample = xl.parse(sheet_name, nrows=2)
            file_size = f"{round(os.path.getsize(file)/(1024*1024), 2)} MB"
            summary_data.append({
                'City': city_name,
                'Sheet_Name': sheet_name,
                'File_Type': 'Excel (XLSX)',
                'File_Size': file_size,
                'Rows': xl.parse(sheet_name).shape[0],
                'Columns': xl.parse(sheet_name).shape[1],
                'Columns_List': df_sample.columns.tolist(),
                'Dtypes': df_sample.dtypes.to_dict()
            })
    except Exception as e:
        print(f"Error reading {file}: {str(e)}")

In [ ]:

# Check for consistency across files:
# Compare columns across all cities
all_columns = summary_df['Columns_List'].explode().unique()
print("Unique columns across all files:", all_columns)

In [ ]:
# Merge all files into a single DataFrame:

all_cities = pd.concat(
    [pd.read_excel(file).assign(City=os.path.splitext(file)[0]) 
    for file in city_files
])

**1. Data Import and Structuring**

**a) Import and Concatenate City Datasets**

In [ ]:
import pandas as pd
import os
import json
from glob import glob

def process_city_file(file_path, city_name):
    """Process a single city file and return structured DataFrame"""
    df = pd.read_csv(file_path)  # or pd.read_excel() if Excel files
    
    # Extract nested JSON data from columns
    for col in ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']:
        df[col] = df[col].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    
    # Flatten nested JSON structures
    details = pd.json_normalize(df['new_car_detail'])
    overview = pd.json_normalize(df['new_car_overview'].apply(lambda x: x['top']))
    features = pd.json_normalize(df['new_car_feature'].apply(lambda x: [item['value'] for item in x['top']]))
    specs = pd.json_normalize(df['new_car_specs'].apply(lambda x: x['top']))
    
    # Combine all data
    processed_df = pd.concat([
        df[['car_links']],
        details,
        overview,
        features,
        specs
    ], axis=1)
    
    # Add city column
    processed_df['City'] = city_name
    
    return processed_df

def process_all_cities(data_dir):
    """Process all city files in directory"""
    all_cities = []
    
    # Get all city files (assuming naming pattern like 'Chennai.csv', 'Mumbai.csv')
    city_files = glob(os.path.join(data_dir, '*.csv'))  # or '*.xlsx' for Excel
    
    for file_path in city_files:
        city_name = os.path.basename(file_path).split('.')[0]
        city_df = process_city_file(file_path, city_name)
        all_cities.append(city_df)
    
    # Concatenate all city DataFrames
    combined_df = pd.concat(all_cities, ignore_index=True)
    
    return combined_df

# Usage:
data_directory = 'path/to/your/city/files'
combined_data = process_all_cities(data_directory)

**2. Handling Missing Values**

In [ ]:
def handle_missing_values(df):
    """Handle missing values in the dataset"""
    
    # Numerical columns - fill with median
    numerical_cols = ['km', 'modelYear', 'ownerNo', 'priceActual']
    for col in numerical_cols:
        if col in df.columns:
            # Clean numerical columns first (remove commas, non-numeric chars)
            df[col] = pd.to_numeric(df[col].astype(str).str.replace('[^\d.]', '', regex=True), errors='coerce')
            df[col].fillna(df[col].median(), inplace=True)
    
    # Categorical columns - fill with mode or 'Unknown'
    categorical_cols = ['ft', 'bt', 'transmission', 'owner', 'oem', 'model']
    for col in categorical_cols:
        if col in df.columns:
            df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else 'Unknown', inplace=True)
    
    # Price handling - extract numerical value from string
    if 'price' in df.columns:
        df['price'] = df['price'].str.extract(r'₹\s*([\d,.]+)')[0]
        df['price'] = pd.to_numeric(df['price'].str.replace(',', ''), errors='coerce')
        df['price'].fillna(df['price'].median(), inplace=True)
    
    return df

# Apply missing value handling
cleaned_data = handle_missing_values(combined_data)

**Standardizing Data Formats**

In [ ]:
def standardize_data_formats(df):
    """Standardize data formats across columns"""
    
    # Convert kms driven to numeric
    if 'km' in df.columns:
        df['km'] = df['km'].astype(str).str.replace('Kms', '').str.replace(',', '').str.strip()
        df['km'] = pd.to_numeric(df['km'], errors='coerce')
    
    # Convert engine displacement to numeric (cc)
    if 'engineDisplacement' in df.columns:
        df['engineDisplacement_cc'] = df['engineDisplacement'].str.extract(r'(\d+)').astype(float)
    
    # Extract power values
    if 'maxPower' in df.columns:
        df['maxPower_bhp'] = df['maxPower'].str.extract(r'([\d.]+)').astype(float)
    
    # Convert torque to numeric
    if 'torque' in df.columns:
        df['torque_nm'] = df['torque'].str.extract(r'([\d.]+)').astype(float)
    
    # Convert year columns to datetime
    year_cols = ['modelYear', 'registrationYear']
    for col in year_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%Y', errors='coerce').dt.year
    
    # Standardize owner information
    if 'owner' in df.columns:
        df['owner'] = df['owner'].str.replace(r'\d(st|nd|rd|th)\s*Owner', lambda m: m.group(1) + ' Owner', regex=True)
    
    return df

# Apply standardization
standardized_data = standardize_data_formats(cleaned_data)

**Feature Engineering**

In [ ]:
def perform_feature_engineering(df):
    """Create new features from existing data"""
    
    # Calculate car age
    current_year = pd.Timestamp.now().year
    if 'modelYear' in df.columns:
        df['car_age'] = current_year - df['modelYear']
    
    # Create binary features from car features
    if 'features' in df.columns:
        all_features = set()
        for feature_list in df['features']:
            if isinstance(feature_list, list):
                all_features.update(feature_list)
        
        for feature in all_features:
            df[f'has_{feature}'] = df['features'].apply(lambda x: feature in x if isinstance(x, list) else False)
    
    # Create transmission type binary columns
    if 'transmission' in df.columns:
        df['is_automatic'] = df['transmission'].str.contains('Automatic', case=False)
        df['is_manual'] = df['transmission'].str.contains('Manual', case=False)
    
    return df

# Apply feature engineering
final_data = perform_feature_engineering(standardized_data)

**5. Saving Processed Data**

In [ ]:
# Save the processed data
final_data.to_csv('processed_car_data.csv', index=False)
# Or save to pickle for preserving data types
final_data.to_pickle('processed_car_data.pkl')

**4. Encoding Categorical Variables**

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

def encode_categorical_variables(df):
    """Encode categorical variables using appropriate techniques"""
    
    # Identify categorical columns
    nominal_cats = ['oem', 'model', 'City', 'fuelType', 'transmissionType']  # No inherent ordering
    ordinal_cats = ['owner']  # Has meaningful order (1st > 2nd > 3rd Owner)
    
    # One-hot encoding for nominal variables
    onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    onehot_encoded = onehot_encoder.fit_transform(df[nominal_cats])
    
    # Create DataFrame from one-hot encoded data
    onehot_df = pd.DataFrame(
        onehot_encoded,
        columns=onehot_encoder.get_feature_names_out(nominal_cats)
    )
    
    # Ordinal encoding for ordered categories
    owner_order = ['4th Owner', '3rd Owner', '2nd Owner', '1st Owner']  # Lower is better
    ordinal_encoder = OrdinalEncoder(categories=[owner_order])
    df['owner_encoded'] = ordinal_encoder.fit_transform(df[['owner']])
    
    # Label encoding for binary categories (if any)
    binary_cols = ['is_automatic', 'is_manual']  # Example binary columns
    for col in binary_cols:
        if col in df.columns:
            df[col] = LabelEncoder().fit_transform(df[col])
    
    # Combine encoded data with original numerical data
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    final_df = pd.concat([
        df[numerical_cols],
        onehot_df,
        df[['owner_encoded']]
    ], axis=1)
    
    return final_df, onehot_encoder, ordinal_encoder

# Apply encoding
encoded_data, ohe, ord_enc = encode_categorical_variables(final_data)

**5. Normalizing Numerical Features**

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def normalize_numerical_features(df, scaling_method='minmax'):
    """Normalize numerical features to common scale"""
    
    # Select numerical columns (excluding target and encoded columns)
    numerical_cols = [col for col in df.columns 
                    if df[col].dtype in ['int64', 'float64'] 
                    and col != 'price'  # Exclude target variable
                    and not col.endswith('_encoded')]
    
    # Apply selected scaling method
    if scaling_method == 'minmax':
        scaler = MinMaxScaler()
    else:  # standard
        scaler = StandardScaler()
    
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    
    return df, scaler

# Apply normalization (choose either 'minmax' or 'standard')
normalized_data, feature_scaler = normalize_numerical_features(encoded_data, 'minmax')

**6. Removing Outliers**

In [ ]:
from scipy import stats

def handle_outliers(df, method='iqr', threshold=1.5):
    """Identify and handle outliers in numerical features"""
    
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    
    if method == 'iqr':
        # IQR method
        for col in numerical_cols:
            if col != 'price':  # Don't remove based on target variable
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - threshold * IQR
                upper_bound = Q3 + threshold * IQR
                
                # Cap outliers instead of removing
                df[col] = df[col].clip(lower_bound, upper_bound)
    
    elif method == 'zscore':
        # Z-score method
        for col in numerical_cols:
            if col != 'price':
                z_scores = stats.zscore(df[col])
                df = df[(np.abs(z_scores) < threshold)]
    
    return df

# Apply outlier handling (choose either 'iqr' or 'zscore')
cleaned_data = handle_outliers(normalized_data, 'iqr', 1.5)

**Complete Data Processing Pipeline**

In [ ]:
def full_data_processing_pipeline(data_dir, city_files):
    """Complete data processing pipeline"""
    
    # 1. Import and concatenate city datasets
    combined_data = process_all_cities(data_dir, city_files)
    
    # 2. Handle missing values
    cleaned_data = handle_missing_values(combined_data)
    
    # 3. Standardize data formats
    standardized_data = standardize_data_formats(cleaned_data)
    
    # 4. Feature engineering
    final_data = perform_feature_engineering(standardized_data)
    
    # 5. Encode categorical variables
    encoded_data, ohe, ord_enc = encode_categorical_variables(final_data)
    
    # 6. Normalize numerical features
    normalized_data, feature_scaler = normalize_numerical_features(encoded_data)
    
    # 7. Handle outliers
    processed_data = handle_outliers(normalized_data)
    
    return processed_data, ohe, ord_enc, feature_scaler

# Example usage:
data_dir = 'path/to/city/files'
city_files = ['Chennai.csv', 'Mumbai.csv', 'Delhi.csv']  # etc.
final_processed_data, ohe, ord_enc, scaler = full_data_processing_pipeline(data_dir, city_files)